<a href="https://colab.research.google.com/github/nueveonce/Etapa3DS/blob/main/Etapa3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>